In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import pandas as pd
import geopandas as gpd
import rasterio
from nso_ds_classes.nso_ds_normalize_scaler import scaler_class_all
from annotations.data_preparation import extract_dataframe_pixels_values_from_tif_and_polygons
from annotations.utils import get_scaler_filepath
from annotations.data_loader import load_annotations_polygons, load_annotations_polygons_gpkg
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import settings_blob

In [ ]:
# function to upload the annotations to the cloud.
def upload_to_blob(apath, directory_blob=""):

    container_name = "satellite-images-nso"
    blob_name = directory_blob+os.path.basename(apath) 


    blob_service_client = BlobServiceClient.from_connection_string(settings_blob.connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)

    with open(apath, "rb") as data:
            blob_client.upload_blob(data ,  overwrite=True)


## Transform Polygon Annotations to Pixel Annotation Parquet files

This script is intended to transform given polygon annotations in geojson (made in i.e. QGis) into pixel level annotations, with scaled band values. The pixel level annotations are written to parquet files.
Change the variables below to match the situation on your device.
Note that these transformations are quite quickly very memory intensive.

Date: 2024-01-11 \
Author: Pieter Kouyzer


In [ ]:
# Set variables
location = "Voornes_Duin"

if location == "Voornes_Duin":
    satellite = "PNEO"
    images_folder = "E:/Data/remote_sensing/satellite-images/PNEO_30CM/Voornes Duincv /"
    regex = "*_height_asphalt_crop.tif"
    annotations_folder = "C:/Users/pzhadmin/Data/remote-sensing/annotations/"
    annotations_polygon_filename_regex = "Voornes Duin PNEO_30CM Annotations_2024-01-19.geojson"
    scaler_folder_path = "../../scalers/"
    col_names = ["r", "g", "b", "n", "e", "d", "ndvi", "re_ndvi", "height"]
    pixel_filepath = os.path.join(annotations_folder, f"{location}_{satellite}_pixel_annotations.parquet")
    pixel_scaled_filepath = os.path.join(annotations_folder, f"{location}_{satellite}_pixel_annotations_scaled.parquet")

elif location == "Coepelduynen":
   
    images_folder = "E:/data/"
    regex = f"{location}/2023*re*asphalt_crop.tif"
    annotations_path = "C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/Annotations_Coepelduynen_2023.gpkg"


### Prepare data

In [ ]:
if location == "Voornes_Duin":
    annotations_polygons_gdf = load_annotations_polygons(annotations_folder, annotations_polygon_filename_regex, regex, images_folder)

elif location == "Coepelduynen":
    annotations_polygons_gdf = load_annotations_polygons_gpkg("C:/repos/satellite-images-nso-datascience/data/annotations/Coepelduynen/Annotations_Coepelduynen_2023.gpkg")
    annotations_polygons_gdf = annotations_polygons_gdf.reset_index(drop=True)
    # Custom actions to set data straight.
    annotations_polygons_gdf.loc[annotations_polygons_gdf["name"] != "Annotations_Coepelduynen_2023","label"] = annotations_polygons_gdf[annotations_polygons_gdf["name"] != "Annotations_Coepelduynen_2023"]["Label_name"]
    annotations_polygons_gdf = annotations_polygons_gdf.drop(["Label_name"], axis=1)
    annotations_polygons_gdf['label'] = annotations_polygons_gdf['label'].str.replace("\nAsphalt","Asphalt")
    annotations_polygons_gdf["Label"] = annotations_polygons_gdf["label"]
    annotations_polygons_gdf = annotations_polygons_gdf.drop(["label"], axis=1)

In [ ]:
if location == "Voornes_Duin":
    if os.path.isfile(pixel_filepath):
        df = pd.read_parquet(pixel_filepath)
    else:
        dfs = []
        for tif_file in glob.glob(os.path.join(images_folder, regex)):
            tif_file = tif_file.replace("\\","/")
            print(tif_file)
            name_tif_file = tif_file.split("/")[-1].split(".")[0]
            with rasterio.open(tif_file) as dataset:
                dfs += [
                    extract_dataframe_pixels_values_from_tif_and_polygons(
                        tif_dataset=dataset, polygon_gdf=annotations_polygons_gdf,name_tif_file=name_tif_file
                    )
                ]
            
        df = pd.concat(dfs)
        df.to_parquet(pixel_filepath)

elif location == "Coepelduynen":
    #Custom aggregation for annnotations across all satellite images
    dfs = []
    for tif_file in glob.glob(os.path.join(images_folder, regex)):
            tif_file = tif_file.replace("\\","/")
            print(tif_file)
            name_tif_file = tif_file.split("/")[-1].split(".")[0]
            with rasterio.open(tif_file) as dataset:
                dfs += [
                extract_dataframe_pixels_values_from_tif_and_polygons(
                            tif_dataset=dataset,
                            polygon_gdf=annotations_polygons_gdf[
                                annotations_polygons_gdf["name"] == "Annotations_Coepelduynen_2023"
                            ],
                            name_tif_file=tif_file.split("/")[-1],
                            name_annotations="Annotations_Coepelduynen_2023",
                        )          
                ]
            
    df = pd.concat(dfs)

    # Annotations for specific satellite images
    dfs = []
    for tif_file in glob.glob(os.path.join(images_folder, regex)):
        tif_file = tif_file.replace("\\","/")
        print(tif_file)
        name_tif_file = tif_file.split("/")[-1].split(".")[0].split("_")[0]+"_annotations"
        print(name_tif_file)
        with rasterio.open(tif_file) as dataset:
                dfs += [
                extract_dataframe_pixels_values_from_tif_and_polygons(
                            tif_dataset=dataset,
                            polygon_gdf=annotations_polygons_gdf[
                                annotations_polygons_gdf["name"] == name_tif_file
                            ],
                            name_tif_file=tif_file.split("/")[-1],
                            name_annotations=name_tif_file,
                        )          
                ]
        
    
    dfs = pd.concat(dfs)
    df = pd.concat([dfs,df])
    
    # Extra check to see if all the correct bands are found
    #if "ndvi" not in df.columns:

    #    if "n" in df.columns:
    #        df['ndvi'] =df.apply(lambda x:(((x['n'] - x['r']) / (x['n'] + x['r']))*100) + 100, axis =1)
    #    if "i" in df.columns:
    #        df['ndvi'] =df.apply(lambda x:(((x['i'] - x['r']) / (x['i'] + x['r']))*100) + 100, axis =1)
    #    else:
    #        print("No infrared band found for ndvi")

    #if "ndwi" not in df.columns:
    #    if "n" in df.columns:
    #        df['ndwi'] = df.apply(lambda x: ((x['g']- x['n'])/(x['n']+x['g'])*100)+100, axis=1 )
    #    if "i" in df.columns:
    #        df['ndwi'] = df.apply(lambda x: ((x['g']- x['i'])/(x['i']+x['g'])*100)+100, axis=1 )
    #    else:
    #        print("No infrared band found for ndwi")

    #if "re_ndvi" not in df.columns:
    #    if "e" in df.columns:
    #        df['re_ndvi'] =df.apply(lambda x:(((x['e'] - x['r']) / (x['e'] + x['r']))*100) + 100, axis =1)
    #    else:
    #        print("")

    df.to_parquet("annotations_pixel_dataframes/annotaties_coepelduynen_to_pixel_2023.parquet")

# Nieuwkoopse Plassen annotations.

In [ ]:
annotation_folder_path = "C:/repos/satellite-images-nso-datascience/data/annotations/Schippergat/{to_replace}*.geojson"

In [ ]:
# PNEO annotations
dfs = []
for annotation_file in glob.glob(annotation_folder_path.replace("{to_replace}", "2023")):
    annotation_file = annotation_file.replace("\\","/")
    print(annotation_file)

    tif_file = glob.glob("E:/data/nieuwkoopse_plassen_schippersgat/"+annotation_file.split("/")[-1].split("_")[0]+"*")[0].replace("\\","/")
    print(tif_file)

    name_tif_file = tif_file.split("/")[-1].split(".")[0].split("_")[0]+"_annotations"
    print(name_tif_file)

    waterplanten_annotations = gpd.read_file(annotation_file)
    waterplanten_annotations["name"] = name_tif_file
    try: 
        waterplanten_annotations["Label"] = waterplanten_annotations["class"]
    except Exception as e:
        print(e)

    waterplanten_annotations  = waterplanten_annotations.to_crs("EPSG:28992")

    
    with rasterio.open(tif_file) as dataset:
         dfs += [extract_dataframe_pixels_values_from_tif_and_polygons(dataset, waterplanten_annotations, tif_file.split("/")[-1], name_tif_file)]

df = pd.concat(dfs)
# TODO: update the descriptions
#df['ndwi'] =df['height']
#df = df.drop(['height'], axis=1)
df.to_parquet("./annotations_pixel_dataframes/PNEO_waterplanten_annotations.parquet")

In [ ]:
df

In [ ]:
# Upload annotations to the cloud
upload_to_blob("./annotations_pixel_dataframes/PNEO_waterplanten_annotations.parquet", "Schippersgat/")

In [ ]:
# Superview annotations
dfs = []
for annotation_file in glob.glob(annotation_folder_path.replace("{to_replace}", "2022"))+  glob.glob(annotation_folder_path.replace("{to_replace}", "2021"))+  glob.glob(annotation_folder_path.replace("{to_replace}", "2019")):
    annotation_file = annotation_file.replace("\\","/")
    print(annotation_file)

    tif_file = glob.glob("E:/data/nieuwkoopse_plassen_schippersgat/"+annotation_file.split("/")[-1].split("_")[0]+"*")[0].replace("\\","/")
    print(tif_file)

    name_tif_file = tif_file.split("/")[-1].split(".")[0].split("_")[0]+"_annotations"
    print(name_tif_file)

    waterplanten_annotations = gpd.read_file(annotation_file)
    waterplanten_annotations["name"] = name_tif_file

    if "label" in waterplanten_annotations.columns:
        waterplanten_annotations["Label"] = waterplanten_annotations["label"]

    try: 
        waterplanten_annotations["Label"] = waterplanten_annotations["class"]
    except Exception as e:
        print(e)

    waterplanten_annotations  = waterplanten_annotations.to_crs("EPSG:28992")

    
    with rasterio.open(tif_file) as dataset:
         dfs += [extract_dataframe_pixels_values_from_tif_and_polygons(dataset, waterplanten_annotations, tif_file.split("/")[-1], name_tif_file )]

In [ ]:
df = pd.concat(dfs)
#TODO: Height variable name is not correct
#df['ndwi'] =df['height']
#df = df.drop(['height'], axis=1)

In [ ]:
df['label'] = df['label'].replace("Waterplanten", "Waterplants")

In [ ]:
df.drop(["image","season","annotation_no", "date"], axis=1).groupby(["label"]).mean()

In [ ]:
df.to_parquet("./annotations_pixel_dataframes/Superview_waterplanten_annotations.parquet")

In [ ]:
# Upload annotations to the cloud
upload_to_blob("./annotations_pixel_dataframes/Superview_waterplanten_annotations.parquet", "Schippersgat/")